In [63]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import RobustScaler
import plotly.express as px
from sklearn.impute import KNNImputer
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [64]:
tickers = pd.read_excel('/content/drive/MyDrive/stock/tickers.xlsx', header=None)
tickers = tickers[0].tolist()
market = yf.download('SPY', start="2022-01-01")
stock_prices = yf.download(tickers, start="2022-01-01")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  100 of 100 completed


In [65]:
stock_close_prices = stock_prices['Close']
stock_close_prices

Ticker,AAL,AAPL,ABEV,ACHR,AEG,AEO,AGNC,ALLY,AMCR,AMD,...,TEVA,TSLA,U,UBER,VALE,VTRS,WBA,WBD,WRD,X
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,18.750000,178.645660,2.300424,6.17,4.264992,23.094658,9.455685,42.862617,10.207474,150.240005,...,8.330000,399.926666,138.649994,43.950001,10.599700,12.160658,43.733078,25.330000,NaN,23.364923
2022-01-04,19.020000,176.378342,2.291967,5.99,4.375882,23.286579,9.523005,43.886097,10.370727,144.419998,...,8.390000,383.196655,129.729996,44.419998,10.400141,12.297584,44.079250,26.750000,NaN,24.097210
2022-01-05,18.680000,171.686722,2.224308,5.43,4.367352,22.409224,9.278198,42.915546,10.387910,136.149994,...,8.300000,362.706665,121.010002,43.240002,10.484570,12.357489,44.507839,26.450001,NaN,24.663517
2022-01-06,18.570000,168.820679,2.198935,5.04,4.495302,22.272135,9.302677,43.639046,10.301990,136.229996,...,8.520000,354.899994,119.750000,42.029999,10.753207,12.408835,43.222057,25.719999,NaN,24.185085
2022-01-07,19.280001,168.987549,2.190477,4.64,4.734142,21.888292,9.363878,44.036095,10.413687,132.000000,...,8.850000,342.320007,120.379997,41.509998,11.367239,12.614221,44.375961,30.059999,NaN,24.800207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-09,11.760000,201.449997,2.480000,11.26,7.260000,10.360000,9.210000,36.599998,9.180000,121.730003,...,18.070000,308.579987,24.690001,87.120003,9.580000,8.810000,11.330000,9.530000,9.15,53.139999
2025-06-10,11.840000,202.669998,2.450000,11.38,7.210000,10.390000,9.330000,36.790001,9.240000,123.239998,...,17.590000,326.089996,24.840000,86.389999,9.660000,9.040000,11.350000,10.010000,8.92,53.889999
2025-06-11,11.060000,198.779999,2.480000,12.17,7.200000,10.110000,9.320000,37.400002,9.160000,121.139999,...,17.320000,326.429993,24.230000,86.580002,9.610000,8.980000,11.340000,10.510000,8.78,53.480000


In [66]:
imputer = KNNImputer(n_neighbors=2)
columns = stock_close_prices.columns
stock_close_prices_imputed = imputer.fit_transform(stock_close_prices)
stock_close_prices = pd.DataFrame(stock_close_prices, columns=columns)
stock_close_prices

Ticker,AAL,AAPL,ABEV,ACHR,AEG,AEO,AGNC,ALLY,AMCR,AMD,...,TEVA,TSLA,U,UBER,VALE,VTRS,WBA,WBD,WRD,X
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-03,18.750000,178.645660,2.300424,6.17,4.264992,23.094658,9.455685,42.862617,10.207474,150.240005,...,8.330000,399.926666,138.649994,43.950001,10.599700,12.160658,43.733078,25.330000,NaN,23.364923
2022-01-04,19.020000,176.378342,2.291967,5.99,4.375882,23.286579,9.523005,43.886097,10.370727,144.419998,...,8.390000,383.196655,129.729996,44.419998,10.400141,12.297584,44.079250,26.750000,NaN,24.097210
2022-01-05,18.680000,171.686722,2.224308,5.43,4.367352,22.409224,9.278198,42.915546,10.387910,136.149994,...,8.300000,362.706665,121.010002,43.240002,10.484570,12.357489,44.507839,26.450001,NaN,24.663517
2022-01-06,18.570000,168.820679,2.198935,5.04,4.495302,22.272135,9.302677,43.639046,10.301990,136.229996,...,8.520000,354.899994,119.750000,42.029999,10.753207,12.408835,43.222057,25.719999,NaN,24.185085
2022-01-07,19.280001,168.987549,2.190477,4.64,4.734142,21.888292,9.363878,44.036095,10.413687,132.000000,...,8.850000,342.320007,120.379997,41.509998,11.367239,12.614221,44.375961,30.059999,NaN,24.800207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-09,11.760000,201.449997,2.480000,11.26,7.260000,10.360000,9.210000,36.599998,9.180000,121.730003,...,18.070000,308.579987,24.690001,87.120003,9.580000,8.810000,11.330000,9.530000,9.15,53.139999
2025-06-10,11.840000,202.669998,2.450000,11.38,7.210000,10.390000,9.330000,36.790001,9.240000,123.239998,...,17.590000,326.089996,24.840000,86.389999,9.660000,9.040000,11.350000,10.010000,8.92,53.889999
2025-06-11,11.060000,198.779999,2.480000,12.17,7.200000,10.110000,9.320000,37.400002,9.160000,121.139999,...,17.320000,326.429993,24.230000,86.580002,9.610000,8.980000,11.340000,10.510000,8.78,53.480000


Calculating BETA

In [67]:
market_returns = market['Close'].pct_change()

In [68]:
returns = stock_close_prices.pct_change()
returns[1:]

Ticker,AAL,AAPL,ABEV,ACHR,AEG,AEO,AGNC,ALLY,AMCR,AMD,...,TEVA,TSLA,U,UBER,VALE,VTRS,WBA,WBD,WRD,X
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-04,0.014400,-0.012692,-0.003677,-0.029173,0.026000,0.008310,0.007120,0.023878,0.015993,-0.038738,...,0.007203,-0.041833,-0.064335,0.010694,-0.018827,0.011260,0.007916,0.056060,NaN,0.031341
2022-01-05,-0.017876,-0.026600,-0.029520,-0.093489,-0.001949,-0.037676,-0.025707,-0.022115,0.001657,-0.057264,...,-0.010727,-0.053471,-0.067216,-0.026565,0.008118,0.004871,0.009723,-0.011215,NaN,0.023501
2022-01-06,-0.005889,-0.016693,-0.011407,-0.071823,0.029297,-0.006118,0.002638,0.016859,-0.008271,0.000588,...,0.026506,-0.021523,-0.010412,-0.027983,0.025622,0.004155,-0.028889,-0.027599,NaN,-0.019398
2022-01-07,0.038234,0.000988,-0.003846,-0.079365,0.053131,-0.017234,0.006579,0.009098,0.010842,-0.031050,...,0.038732,-0.035447,0.005261,-0.012372,0.057102,0.016552,0.026697,0.168740,NaN,0.025434
2022-01-10,-0.025415,0.000116,0.003861,-0.094828,-0.003604,-0.018372,0.002615,0.005009,0.002475,0.000000,...,0.030508,0.030342,0.046686,0.026259,-0.010804,0.018318,-0.002043,-0.048902,NaN,-0.027953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-09,-0.001698,-0.012113,-0.004016,0.105005,0.001379,0.019685,0.009868,0.008264,0.007684,0.047681,...,0.016882,0.045538,-0.017509,0.017757,0.009484,0.011481,0.007111,-0.029532,0.037415,-0.004869
2025-06-10,0.006803,0.006056,-0.012097,0.010657,-0.006887,0.002896,0.013029,0.005191,0.006536,0.012404,...,-0.026563,0.056744,0.006075,-0.008379,0.008351,0.026107,0.001765,0.050367,-0.025137,0.014114
2025-06-11,-0.065878,-0.019194,0.012245,0.069420,-0.001387,-0.026949,-0.001072,0.016581,-0.008658,-0.017040,...,-0.015350,0.001043,-0.024557,0.002199,-0.005176,-0.006637,-0.000881,0.049950,-0.015695,-0.007608


In [69]:
betas = {}
for stock in tickers:
    stock_returns_array = returns[stock][1:].values.reshape(-1, 1)
    market_returns_array = market_returns[1:].values.reshape(-1, 1)
    cov = np.cov(stock_returns_array, market_returns_array, rowvar=False)
    beta = cov[0, 1] / cov[1, 1]
    betas[stock] = beta

In [70]:
betas

{'NVDA': np.float64(2.1653234635863234),
 'LCID': np.float64(1.7414121304977528),
 'PLTR': np.float64(2.137432024654458),
 'F': np.float64(1.2294546631275967),
 'TSLA': np.float64(2.0141301469253965),
 'INTC': np.float64(1.4088317418926606),
 'VTRS': np.float64(0.7770796233245694),
 'BBD': np.float64(0.5619224189224782),
 'SOFI': np.float64(2.0819077133801045),
 'QBTS': np.float64(1.317910452270533),
 'GAP': np.float64(1.493341258150867),
 'AAL': np.float64(1.5301278036569084),
 'NU': np.float64(1.5765476061244827),
 'IPG': np.float64(0.9342198040419332),
 'PATH': np.float64(1.9526323866164284),
 'MRVL': np.float64(2.292522810015999),
 'RGTI': np.float64(2.1454684206207784),
 'U': np.float64(2.4589516812200083),
 'VALE': np.float64(0.6549088054271912),
 'BTG': np.float64(0.6347888060237394),
 'AAPL': np.float64(1.237339055165864),
 'HIMS': np.float64(1.8097609374887844),
 'ITUB': np.float64(0.5552962378004695),
 'WBD': np.float64(1.4639023182133133),
 'AMCR': np.float64(0.7038119601928

In [71]:
beta_df = pd.DataFrame.from_dict(betas, orient='index', columns=['Beta'])
print(beta_df)

          Beta
NVDA  2.165323
LCID  1.741412
PLTR  2.137432
F     1.229455
TSLA  2.014130
...        ...
CPRT  0.971764
AEG   0.919810
BABA  1.043178
MGM   1.440503
KMI   0.616546

[100 rows x 1 columns]


Risk vs. Return

In [72]:
risk_free_rate = 0.02 # Example: 2% annual risk-free rate

stats = pd.DataFrame(index=tickers)
stock_returns = returns[tickers]
stats['Mean Return'] = returns[tickers].mean() * 252
stats['Volatility'] = returns[tickers].std() * (252 ** 0.5)


import plotly.express as px

fig = px.scatter(stats,
                 x='Volatility',
                 y='Mean Return',
                 text=stats.index,
                 title='Interactive Risk vs. Return (Annualized)')

fig.update_traces(textposition='top center')
fig.show()

In [73]:

stats['Sharpe'] = np.nan
stats['CVaR_95'] = np.nan
stats['Max_Drawdown'] = np.nan


daily_rf = risk_free_rate / 252


for stock in tickers:
    ret = stock_returns[stock]
    excess_ret = ret - daily_rf

    sharpe = (excess_ret.mean() / excess_ret.std()) * np.sqrt(252)
    stats.loc[stock, 'Sharpe'] = sharpe

    if not ret.dropna().empty:
        sorted_returns = np.sort(ret.dropna())
        var_index = int(0.05 * len(sorted_returns))
        if var_index < len(sorted_returns):
            cvar = sorted_returns[:var_index].mean()
            stats.loc[stock, 'CVaR_95'] = cvar
        else:
            stats.loc[stock, 'CVaR_95'] = np.nan
    else:
        stats.loc[stock, 'CVaR_95'] = np.nan

    cum_returns = (1 + ret.fillna(0)).cumprod()
    peak = cum_returns.cummax()
    drawdown = (cum_returns - peak) / peak
    max_drawdown = drawdown.min()
    stats.loc[stock, 'Max_Drawdown'] = max_drawdown

stats[['Sharpe', 'CVaR_95', 'Max_Drawdown']].sort_values(by='Sharpe', ascending=False)

,Sharpe,CVaR_95,Max_Drawdown
CORZ,1.403044,-0.115261,-0.649479
PLTR,1.139362,-0.088167,-0.676201
HIMS,1.130791,-0.099709,-0.631219
SMCI,1.128743,-0.119199,-0.848410
AVGO,1.075989,-0.057506,-0.411497
...,...,...,...
NIO,-0.520591,-0.094049,-0.906185
LCID,-0.710711,-0.104586,-0.955795
MRNA,-0.727405,-0.086547,-0.899383
WBA,-0.733797,-0.061166,-0.821197


Calculating Risk Score of each stock

In [74]:
stats['Beta'] = beta_df['Beta']
stats = pd.merge(stats, risk_df[['Sharpe', 'CVaR_95', 'Max_Drawdown']], left_index=True, right_index=True, how='left')

z_score = norm.ppf(0.05)

common_tickers = stats.index.intersection(returns.columns)
if not common_tickers.empty:
    mean_returns = returns[common_tickers].mean()
    std_returns = returns[common_tickers].std()
    stats.loc[common_tickers, 'VaR_95'] = mean_returns + z_score * std_returns
else:
    stats['VaR_95'] = np.nan

for stock in tickers:
    t = yf.Ticker(stock).info
    stats.loc[stock, 'P/E'] = t.get('trailingPE', np.nan)
    stats.loc[stock, 'EPS'] = t.get('trailingEps', np.nan)
    stats.loc[stock, 'Volume'] = t.get('averageVolume', np.nan)

weights = {
    'Beta': 0.2,
    'Volatility': 0.1,
    'VaR_95': 0.1,
    'P/E': 0.1,
    'EPS': 0.1,
    'Volume': 0.1,
    'Sharpe': 0.1,
    'CVaR_95': 0.1,
    'Max_Drawdown': 0.1
}
stats['Risk Score'] = sum(weights[col] * normalized[col] for col in weights)

print(stats[['Risk Score']].sort_values(by='Risk Score', ascending=False))

      Risk Score
PLTR    0.614514
NVDA    0.614496
AVGO    0.527148
AAPL    0.503191
TSLA    0.502910
...          ...
MRNA    0.252689
PONY    0.246622
SOUN    0.235677
WBA     0.235516
WRD     0.195030

[100 rows x 1 columns]


Cluster Risk Profiles

In [75]:
portfolio_df = pd.read_csv('/content/drive/MyDrive/stock/user_portfolios.csv')
portfolio_melted = portfolio_df.melt(id_vars=['user_id'], var_name='Ticker', value_name='Shares')
portfolio_melted = portfolio_melted[portfolio_melted['Shares'] > 0]
portfolio_melted

,user_id,Ticker,Shares
0,1,NVDA,2.30
1,2,NVDA,2.60
2,3,NVDA,2.77
3,4,NVDA,1.52
7,8,NVDA,6.39
...,...,...,...
9993,94,KMI,0.44
9995,96,KMI,3.48
9996,97,KMI,6.35
9998,99,KMI,1.41


In [76]:
stats_reset = stats.reset_index().rename(columns={'index': 'Ticker'})
portfolio_with_risk = pd.merge(portfolio_melted, stats_reset[['Ticker', 'Risk Score']], on='Ticker', how='left')

In [77]:
total_shares_per_user = portfolio_with_risk.groupby('user_id')['Shares'].sum().reset_index()
total_shares_per_user.rename(columns={'Shares': 'Total Shares'}, inplace=True)

portfolio_with_risk = pd.merge(portfolio_with_risk, total_shares_per_user, on='user_id', how='left')

portfolio_with_risk['Weight'] = portfolio_with_risk['Shares'] / portfolio_with_risk['Total Shares']

portfolio_with_risk['Weighted Risk Score'] = portfolio_with_risk['Weight'] * portfolio_with_risk['Risk Score']

portfolio_risk_scores = portfolio_with_risk.groupby('user_id')['Weighted Risk Score'].sum().reset_index()

print("Portfolio Risk Scores per User:")
display(portfolio_risk_scores.sort_values(by='Weighted Risk Score', ascending=False))

Portfolio Risk Scores per User:


,user_id,Weighted Risk Score
0,1,0.442670
43,44,0.432981
50,51,0.429938
97,98,0.427957
33,34,0.425453
...,...,...
28,29,0.392507
30,31,0.392008
11,12,0.389250
82,83,0.388734


In [82]:
scaler = RobustScaler()
stats_normalized = pd.DataFrame(scaler.fit_transform(stats), columns=stats.columns, index=stats.index)

features = normalized[['Beta', 'Volatility', 'VaR_95', 'P/E', 'EPS', 'Volume','Sharpe', 'CVaR_95', 'Max_Drawdown']]

kmeans = KMeans(n_clusters=5, random_state=42)
clusters = kmeans.fit_predict(features)

stats_normalized['Cluster'] = clusters

cluster_order = stats_normalized.groupby('Cluster')['Risk Score'].mean().sort_values().index
cluster_map = {old: new for new, old in enumerate(cluster_order)}
stats_normalized['Cluster'] = stats_normalized['Cluster'].map(cluster_map)

In [83]:
fig = px.scatter(stats_normalized,
                 x='Volatility',
                 y='Risk Score',
                 color='Cluster',
                 text=stats.index,
                 title='Composite Risk Score')

fig.update_traces(textposition='top center')
fig.show()

In [84]:
dbscan = DBSCAN(eps=0.2, min_samples=10)
clusters = dbscan.fit_predict(features)
stats_normalized['DBSCAN_Cluster'] = clusters

cluster_order = stats_normalized.groupby('DBSCAN_Cluster')['Risk Score'].mean().sort_values().index
cluster_map = {old: new for new, old in enumerate(cluster_order)}
stats_normalized['DBSCAN_Cluster'] = stats_normalized['DBSCAN_Cluster'].map(cluster_map)

In [85]:
fig = px.scatter(stats_normalized,
                 x='Volatility',
                 y='Risk Score',
                 color='DBSCAN_Cluster',
                 text=stats.index,
                 title='Composite Risk Score')

fig.update_traces(textposition='top center')
fig.show()